In [1]:
# Data analysis and manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Loadind dataset
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

target = train_df['Transported']

complete_df = pd.concat([train_df.drop('Transported', axis=1), test_df], axis=0, ignore_index=True)

# EDA - Spaceship Dataset

### Variables Qualitatives:
* Nominal: Name, PassengerId, HomePlanet, CryoSleep, Cabin, Destination, VIP, Transported
* Ordinal: -
 
### Variables Quantitatives:
* Continuous: RoomService, FoodCourt, ShoppingMall, Spa, VRDeck
* Discrete: Age

In [2]:
complete_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines


In [3]:
complete_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   12970 non-null  object 
 1   HomePlanet    12682 non-null  object 
 2   CryoSleep     12660 non-null  object 
 3   Cabin         12671 non-null  object 
 4   Destination   12696 non-null  object 
 5   Age           12700 non-null  float64
 6   VIP           12674 non-null  object 
 7   RoomService   12707 non-null  float64
 8   FoodCourt     12681 non-null  float64
 9   ShoppingMall  12664 non-null  float64
 10  Spa           12686 non-null  float64
 11  VRDeck        12702 non-null  float64
 12  Name          12676 non-null  object 
dtypes: float64(6), object(7)
memory usage: 1.3+ MB


In [4]:
complete_df.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,12700.000000,12707.000000,12681.000000,12664.000000,12686.000000,12702.000000
mean,28.771969,222.897852,451.961675,174.906033,308.476904,306.789482
std,14.387261,647.596664,1584.370747,590.558690,1130.279641,1180.097223
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,49.000000,77.000000,29.000000,57.000000,42.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [5]:
print(f'Dataset Nulls:\n\n{complete_df.isnull().sum()}')

Dataset Nulls:

PassengerId       0
HomePlanet      288
CryoSleep       310
Cabin           299
Destination     274
Age             270
VIP             296
RoomService     263
FoodCourt       289
ShoppingMall    306
Spa             284
VRDeck          268
Name            294
dtype: int64


### Feature Engineering

In [6]:
# New features based on PassengerID
complete_df['GroupId'] = complete_df['PassengerId'].str.split('_').str[0].astype(int)

group_sizes = complete_df['GroupId'].value_counts()
complete_df['GroupSize'] = complete_df['GroupId'].map(group_sizes)

In [7]:
# New features based on Cabin
complete_df['Cabin'] = complete_df['Cabin'].fillna('Unknown/Unknown/Unknown')

complete_df['CabinDeck'] = complete_df['Cabin'].str.split('/').str[0]
complete_df['CabinNum'] = pd.to_numeric(complete_df['Cabin'].str.split('/').str[1], errors="coerce").fillna(-1).astype(int)
complete_df['CabinSide'] = complete_df['Cabin'].str.split('/').str[2]

# Adding NaN again
complete_df['Cabin'] = complete_df['Cabin'].replace('Unknown/Unknown/Unknown', np.nan)
complete_df['CabinDeck'] = complete_df['CabinDeck'].replace('Unknown', np.nan)
complete_df['CabinSide'] = complete_df['CabinSide'].replace('Unknown', np.nan)
complete_df['CabinNum'] = complete_df['CabinNum'].replace(-1, np.nan)

In [8]:
# New feature with the total spend on services, controlling the NaN
expenses_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

total_exp = complete_df[expenses_cols].fillna(0).sum(axis=1)
all_nan_mask = complete_df[expenses_cols].isna().all(axis=1)
complete_df['TotalExpense'] = total_exp.mask(all_nan_mask, np.nan)

In [9]:
# New feature with just the surname of the people
complete_df['Surname'] = complete_df['Name'].str.split(' ').str[1]

In [10]:
complete_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,GroupId,GroupSize,CabinDeck,CabinNum,CabinSide,TotalExpense,Surname
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,1,1,B,0.0,P,0.0,Ofracculy
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,2,1,F,0.0,S,736.0,Vines
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,3,2,A,0.0,S,10383.0,Susent
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,3,2,A,0.0,S,5176.0,Susent
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,4,1,F,1.0,S,1091.0,Santantines


### Missing Values

#### HomePlanet

In [11]:
# Revisamos si todas las personas del mismo grupo vienen del mismo planeta
group_planet_counts = complete_df.groupby("GroupId")["HomePlanet"].nunique().value_counts().reset_index()

group_planet_counts.columns = ['NumPlanetasDistintos', 'NumGrupos']
group_planet_counts

,NumPlanetasDistintos,NumGrupos
0,1,9124
1,0,156


In [12]:
# Rellenamos el planeta nan de las personas con grupo en el que se conozca algun home
group_planet = complete_df.groupby("GroupId")["HomePlanet"].apply(lambda x: x.dropna().iloc[0] if x.notna().any() else np.nan)

home_mask = complete_df["HomePlanet"].isna() & complete_df["GroupId"].isin(group_planet.dropna().index)
complete_df.loc[home_mask, "HomePlanet"] = complete_df.loc[home_mask, "GroupId"].map(group_planet).replace({None: np.nan})

In [13]:
print(f'HomePlanet Nulls: {complete_df["HomePlanet"].isnull().sum()}')

HomePlanet Nulls: 157


In [14]:
# Revisamos si todas las personas del mismo apellido vienen del mismo planeta
surname_group_counts = complete_df.groupby("Surname")["HomePlanet"].nunique().value_counts().reset_index()

surname_group_counts.columns = ['NumHomesDistintos', 'NumSurnames']
surname_group_counts

,NumHomesDistintos,NumSurnames
0,1,2400
1,0,6


In [15]:
# Rellenamos el planeta nan de las personas con el mismo apellido
surname_planet = complete_df.groupby("Surname")["HomePlanet"].apply(lambda x: x.dropna().iloc[0] if x.notna().any() else np.nan)

surname_mask = complete_df["HomePlanet"].isna() & complete_df["Surname"].notna()
complete_df.loc[surname_mask, "HomePlanet"] = complete_df.loc[surname_mask, "Surname"].map(surname_planet).replace({None: np.nan})

In [16]:
print(f'HomePlanet Nulls: {complete_df["HomePlanet"].isnull().sum()}')

HomePlanet Nulls: 13


In [17]:
# Revisamos si hay decks con solo un HomePlanet y cual es
deck_planet_list = complete_df.groupby("CabinDeck")["HomePlanet"].unique().reset_index()

deck_planet_list.columns = ['CabinDeck', 'Planetas']
deck_planet_list

,CabinDeck,Planetas
0,A,"[Europa, nan]"
1,B,[Europa]
2,C,"[Europa, nan]"
3,D,"[Mars, Europa, nan]"
4,E,"[Earth, Mars, Europa, nan]"
5,F,"[Earth, Mars, nan]"
6,G,"[Earth, nan]"
7,T,[Europa]


In [18]:
# Podemos ver que en el deck A, B, C y T solo hay Europa y en el G solo hay Earth, rellenamos los valores nan
deck_eur_mask = complete_df["HomePlanet"].isna() & complete_df["CabinDeck"].isin(['A', 'B', 'C', 'T'])
complete_df.loc[deck_eur_mask, "HomePlanet"] = 'Europa'

deck_ear_mask = complete_df["HomePlanet"].isna() & (complete_df["CabinDeck"]=='G')
complete_df.loc[deck_ear_mask, "HomePlanet"] = 'Earth'

In [19]:
print(f'HomePlanet Nulls: {complete_df["HomePlanet"].isnull().sum()}')

HomePlanet Nulls: 10


In [20]:
complete_df[complete_df['HomePlanet'].isna()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,GroupId,GroupSize,CabinDeck,CabinNum,CabinSide,TotalExpense,Surname
2274,2443_01,NaN,False,D/72/P,TRAPPIST-1e,31.0,False,1458.0,421.0,76.0,0.0,0.0,NaN,2443,1,D,72.0,P,1955.0,NaN
2631,2817_01,NaN,False,F/584/P,TRAPPIST-1e,25.0,False,237.0,0.0,910.0,0.0,12.0,Sealfs Sutty,2817,1,F,584.0,P,1159.0,Sutty
3091,3331_01,NaN,False,F/631/S,TRAPPIST-1e,40.0,False,666.0,4.0,83.0,0.0,50.0,NaN,3331,1,F,631.0,S,803.0,NaN
4548,4840_01,NaN,True,F/915/S,TRAPPIST-1e,36.0,False,0.0,0.0,NaN,0.0,0.0,NaN,4840,1,F,915.0,S,0.0,NaN
5762,6108_01,NaN,False,F/1166/S,TRAPPIST-1e,13.0,False,206.0,28.0,0.0,1.0,629.0,NaN,6108,1,F,1166.0,S,864.0,NaN
8969,0616_01,NaN,False,E/40/S,TRAPPIST-1e,31.0,False,0.0,428.0,NaN,1154.0,1025.0,Arkaban Spriney,616,1,E,40.0,S,2607.0,Spriney
10583,4052_01,NaN,False,D/136/P,TRAPPIST-1e,26.0,False,6.0,5903.0,0.0,240.0,72.0,Ankabih Coneveseng,4052,1,D,136.0,P,6221.0,Coneveseng
11913,7065_01,NaN,True,E/471/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Zedares Maltorted,7065,1,E,471.0,S,0.0,Maltorted
12551,8435_01,NaN,NaN,F/1730/P,TRAPPIST-1e,19.0,False,162.0,11.0,0.0,0.0,1216.0,NaN,8435,1,F,1730.0,P,1389.0,NaN
12725,8775_01,NaN,True,D/275/P,TRAPPIST-1e,40.0,False,0.0,0.0,0.0,0.0,0.0,Raston Maltorted,8775,1,D,275.0,P,0.0,Maltorted


In [21]:
# Filtramos los pasajeros con destino TRAPPIST por planeta de origen
origin_distribution = complete_df[complete_df['Destination'] == 'TRAPPIST-1e']['HomePlanet'].value_counts(dropna=False).reset_index()

origin_distribution.columns = ['HomePlanet', 'NumPasajeros']
origin_distribution

,HomePlanet,NumPasajeros
0,Earth,4778
1,Mars,2314
2,Europa,1769
3,NaN,10


In [22]:
# Asignamos como planeta origen segun la moda y la distribución de decks
ear_mask = complete_df["HomePlanet"].isna() & complete_df["CabinDeck"].isin(['E', 'F', 'G'])
complete_df.loc[ear_mask, 'HomePlanet'] = 'Earth'

mar_mask = complete_df["HomePlanet"].isna() & complete_df["CabinDeck"].isin(['D'])
complete_df.loc[mar_mask, 'HomePlanet'] = 'Mars'

eur_mask = complete_df["HomePlanet"].isna() & complete_df["CabinDeck"].isin(['A', 'B', 'C', 'T'])
complete_df.loc[eur_mask, 'HomePlanet'] = 'Europa'

In [23]:
print(f'HomePlanet Nulls: {complete_df["HomePlanet"].isnull().sum()}')

HomePlanet Nulls: 0


#### NEXT VARIABLE

In [24]:
# Revisamos si todas las personas del mismo grupo tienen el mismo apellido
group_surnames_counts = complete_df.groupby("GroupId")["Surname"].nunique().value_counts().reset_index()

group_surnames_counts.columns = ['NumSurnamesDistintos', 'NumGrupos']
group_surnames_counts

,NumSurnamesDistintos,NumGrupos
0,1,8759
1,2,322
2,0,155
3,3,43
4,4,1


In [25]:
# Revisamos si todas las personas del mismo apellido tienen el mismo grupo
surname_group_counts = complete_df.groupby("Surname")["GroupId"].nunique().value_counts().reset_index()

surname_group_counts.columns = ['NumGroupsDistintos', 'NumSurnames']
surname_group_counts

,NumGroupsDistintos,NumSurnames
0,2,457
1,3,404
2,1,341
3,4,340
4,5,269
5,6,224
6,7,159
7,8,98
8,9,60
9,10,27


In [26]:
# Revisamos si todas las personas de la misma cabina comparten apellido
surname_cabin_counts = complete_df.groupby("Cabin")["Surname"].nunique().value_counts().reset_index()

surname_cabin_counts.columns = ['NumSurnamesDistintos', 'NumCabinas']
surname_cabin_counts

,NumSurnamesDistintos,NumCabinas
0,1,9322
1,2,287
2,0,181
3,3,35


In [27]:
# Hay grupos y cabinas con distintos apellidos, y apellidos repartidos en distintos grupos

In [28]:
# Revisamos si todas las personas del mismo grupo van hacia el mismo planeta
group_dest_counts = complete_df.groupby('GroupId')['Destination'].nunique().value_counts().reset_index()

group_dest_counts.columns = ['NumDestinosDistintos', 'NumGrupos']
group_dest_counts

# Hay personas del mismo grupo con destinos diferentes

,NumDestinosDistintos,NumGrupos
0,1,8056
1,2,997
2,0,154
3,3,73


In [29]:
# Revisamos si hay cabinas con personas de diferentes grupos
real_cabins = complete_df[complete_df['Cabin'] != 'Unknown/Unknown/Unknown']

cabin_group_counts = real_cabins.groupby('Cabin')['GroupId'].nunique().value_counts().reset_index()

cabin_group_counts.columns = ['NumGruposDistintos', 'NumCabinas']
cabin_group_counts

# Todas las personas en una cabina son del mismo grupo

,NumGruposDistintos,NumCabinas
0,1,9825


In [30]:
# Revisamos si hay grupos con mas de una cabina
group_cabin_counts = complete_df.groupby("GroupId")["Cabin"].nunique().value_counts().reset_index()

group_cabin_counts.columns = ["NumCabinasDistintas", "NumGrupos"]
group_cabin_counts

# Hay grupos con más de una cabina

,NumCabinasDistintas,NumGrupos
0,1,8446
1,2,640
2,0,161
3,3,33


In [31]:
def classify_group_cabin(cryo):
    n_true = (cryo == True).sum()
    n_false = (cryo == False).sum()
    n_nan = cryo.isna().sum()
    
    if n_true > 0 and n_false == 0 and n_nan == 0:
        return "Todos duermen"
    elif n_true == 0 and n_false > 0 and n_nan == 0:
        return "Todos despiertos"
    elif n_true > 0 and n_false > 0 and n_nan == 0:
        return "Durmiendo y despiertos"
    elif n_true > 0 and n_false == 0 and n_nan > 0:
        return "Todos duermen + NaN"
    elif n_true == 0 and n_false > 0 and n_nan > 0:
        return "Todos despiertos + NaN"
    elif n_true > 0 and n_false > 0 and n_nan > 0:
        return "Durmiendo y despiertos + NaN"
    else:
        return "Solo NaN"

In [32]:
# Revisamos si hay grupos con personas durmiendo y despiertas
group_class_nan = complete_df.groupby("GroupId")["CryoSleep"].apply(classify_group_cabin).value_counts().reset_index()

group_class_nan.columns = ['Tipos', 'NumGrupos']
group_class_nan.sort_values(by="NumGrupos", ascending=False).reset_index(drop=True)

,Tipos,NumGrupos
0,Todos despiertos,5327
1,Todos duermen,2480
2,Durmiendo y despiertos,1167
3,Solo NaN,156
4,Durmiendo y despiertos + NaN,63
5,Todos despiertos + NaN,51
6,Todos duermen + NaN,36


In [33]:
# Revisamos si hay cabinas con personas durmiendo y despiertas
real_cabins = complete_df[complete_df["Cabin"] != "Unknown/Unknown/Unknown"]
cabin_class_nan = real_cabins.groupby("Cabin")["CryoSleep"].apply(classify_group_cabin).value_counts().reset_index()

cabin_class_nan.columns = ['Tipos', 'NumCabinas']
cabin_class_nan.sort_values(by="NumCabinas", ascending=False).reset_index(drop=True)

,Tipos,NumCabinas
0,Todos despiertos,5931
1,Todos duermen,2710
2,Durmiendo y despiertos,885
3,Solo NaN,179
4,Durmiendo y despiertos + NaN,49
5,Todos despiertos + NaN,36
6,Todos duermen + NaN,35


In [34]:
# Aunque tanto para los grupos como para las cabinas la mayoria de sus integrantes toman la misma decisión en cuanto al CryoSleep
# Hay grupos y cabinas con personas durmiendo y personas despiertas mezcladas

In [35]:
# If you have expenses is because you are not in cryosleep
expense_nancryo_mask = (complete_df['TotalExpense'] > 0) & complete_df['CryoSleep'].isna()
complete_df.loc[expense_nancryo_mask, 'CryoSleep'] = False

In [36]:
# If you are in cryosleep you can't have expenses
is_cryo_mask = complete_df["CryoSleep"] == True
complete_df.loc[is_cryo_mask, expenses_cols] = complete_df.loc[is_cryo_mask, expenses_cols].fillna(0)
complete_df.loc[is_cryo_mask, 'TotalExpense'] = 0

In [37]:
print(f'Dataset Nulls:\n\n{complete_df.isnull().sum()}')

Dataset Nulls:

PassengerId       0
HomePlanet        0
CryoSleep       136
Cabin           299
Destination     274
Age             270
VIP             296
RoomService     170
FoodCourt       180
ShoppingMall    175
Spa             177
VRDeck          177
Name            294
GroupId           0
GroupSize         0
CabinDeck       299
CabinNum        299
CabinSide       299
TotalExpense      0
Surname         294
dtype: int64


In [38]:
# Encontrar familias por apellido y/o grupoID, seguro que llegan desde el mismo planeta y van hacia el mismo, podria completar cabina tambien
# Mismo grupo y mismo apellido -  Puede que misma cabina, mismo pais de destino e incluso cryo a la vez

In [39]:
complete_df["CabinDeck"].unique()
complete_df["CabinSide"].unique()


array(['P', 'S', nan], dtype=object)